In [1]:
import csv
import cv2
import os
import numpy as np
from random import shuffle
import matplotlib.pyplot as plt
import imutils
from skimage import data, color, exposure
from sklearn import svm
import cPickle as pickle
import data_generator as dg
import multiscale_detect as md
import kmeansutil as ku
%matplotlib inline  

Get data array from generate_data module. Randomize the array. Pick sample_size amount from each

Data is stored in a pickled zipped list of features and labels

In [2]:
#get positive images, determine best ratios, assign to images

pimages = dg.getImagesFromJSON(open("labels.json").read())
clusters, images = ku.addClusterLabels(pimages)
ratios = [r[0] for r in clusters]
ratios.sort()
print ratios

[0.46285263035703317, 0.77338458477358618, 1.1003667227651768]


Separate positive images into aspect ratios, get negative images and crop to aspect ratios

In [3]:
pratio0imgs = []
pratio1imgs = []
pratio2imgs = []

for image in images:
    if image[1] == 0:
        pratio0imgs.append(image[0])
    elif image[1] == 1:
        pratio1imgs.append(image[0])
    elif image[1] == 2:
        pratio2imgs.append(image[0])

print "Number pos imgs at ratio 0: ", len(pratio0imgs)
print "Number pos imgs at ratio 1: ", len(pratio1imgs)
print "Number pos imgs at ratio 2: ", len(pratio2imgs)
        
#get negative images, use ratios found for positive images to match
nimgfiles = dg.getAllFiles("blanks")
nimages = [cv2.imread("blanks/"+ f) for f in nimgfiles]
   
nratio0imgs = [dg.cropToRatio(x, ratios[0]) for x in nimages]
nratio1imgs = [dg.cropToRatio(x, ratios[1]) for x in nimages]
nratio2imgs = [dg.cropToRatio(x, ratios[2]) for x in nimages]

print "Number neg imgs at ratio 0: ", len(nratio0imgs)
print "Number neg imgs at ratio 1: ", len(nratio1imgs)
print "Number pos neg at ratio 2: ", len(nratio2imgs)


Number pos imgs at ratio 0:  53
Number pos imgs at ratio 1:  140
Number pos imgs at ratio 2:  77
Number neg imgs at ratio 0:  1000
Number neg imgs at ratio 1:  1000
Number pos neg at ratio 2:  1000


Generate features from HOGs and images for each label and aspect ratio

In [4]:
def getFeaturesWithLabel(imageData, hog, dims, label):
    data = []
    for img in imageData: 
        img = cv2.resize(img, dims)

        #for images with transparency layer, reduce to 3 layers
        feat = hog.compute(img)
        
        data.append((feat, label))
    return data


In [5]:
minDim = 80
HOGs, dims = dg.ratiosToHOGS(ratios, minDim)


(80, 32)


In [20]:
pdata0ratio = getFeaturesWithLabel(pratio0imgs, HOGs[0], dims[0], 1)
pdata1ratio = getFeaturesWithLabel(pratio1imgs, HOGs[1], dims[1], 1)
pdata2ratio = getFeaturesWithLabel(pratio2imgs, HOGs[2], dims[2], 1)

ndata0ratio = getFeaturesWithLabel(nratio0imgs, HOGs[0], dims[0], 0)
ndata1ratio = getFeaturesWithLabel(nratio1imgs, HOGs[1], dims[1], 0)
ndata2ratio = getFeaturesWithLabel(nratio2imgs, HOGs[2], dims[2], 0)

data0ratio = pdata0ratio + ndata0ratio
data1ratio = pdata1ratio + ndata1ratio
data2ratio = pdata2ratio + ndata2ratio

shuffle(data0ratio)
shuffle(data1ratio)
shuffle(data2ratio)

feat0, label0 = map(list, zip(*data0ratio))
feat1, label1 = map(list, zip(*data1ratio))
feat2, label2 = map(list, zip(*data2ratio))

feat0 = [x.flatten() for x in feat0]
feat1 = [x.flatten() for x in feat1]
feat2 = [x.flatten() for x in feat2]

sample_size0 = len(feat0)
sample_size1 = len(feat1)
sample_size2 = len(feat2)

train_size0 = int(round(.8*sample_size0))
train_size1 = int(round(.8*sample_size1))
train_size2 = int(round(.8*sample_size2))



In [21]:
train_feat0 = np.array(feat0[:train_size0], np.float32)
test_feat0= np.array(feat0[train_size0: sample_size0], np.float32)
train_label0 = np.array(label0[:train_size0])
test_label0 = np.array(label0[train_size0:sample_size0])

train_feat1= np.array(feat1[:train_size1], np.float32)
test_feat1 = np.array(feat1[train_size1: sample_size1], np.float32)
train_label1 = np.array(label1[:train_size1])
test_label1 = np.array(label1[train_size1:sample_size1])

train_feat2 = np.array(feat2[:train_size2], np.float32)
test_feat2 = np.array(feat2[train_size2: sample_size2], np.float32)
train_label2 = np.array(label2[:train_size2])
test_label2 = np.array(label2[train_size2:sample_size2])

Train SVM

In [29]:
lsvm0 = svm.SVC(gamma=5, C=.1, kernel="linear")
lsvm0.fit(train_feat0, train_label0)

lsvm1 = svm.SVC(gamma=5, C=.1, kernel="linear")
lsvm1.fit(train_feat1, train_label1)

lsvm2 = svm.SVC(gamma=5, C=.1, kernel="linear")
lsvm2.fit(train_feat2, train_label2)


SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=5, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Compute and display training accuracy

In [30]:
print lsvm0.score(train_feat0, train_label0)
print lsvm1.score(train_feat1, train_label1)
print lsvm2.score(train_feat2, train_label2)

0.998812351544
1.0
1.0


Compute and display test set accuracy, true and false positives, true and false negatives, list of files that were misclassified

In [36]:
def printConfusionMatrix(result, labels):
    result0 = [int(x) for x in result]

    false_positives = 0
    false_negatives = 0
    true_positives = 0
    true_negatives = 0

    for i, row in enumerate(labels):
        if row  == 1 and result[i] == 1: 
            true_positives +=1
        if row  != 1 and result[i] != 1: 
            true_negatives +=1
        if row  != 1 and result[i] == 1: 
            false_positives +=1
        if row == 1 and result[i] != 1: 
            false_negatives +=1	

    print "true positives:", true_positives, "true_negatives:", true_negatives, "false positives:", false_positives, "false_negatives:", false_negatives, "\n"


#compute test accuracy
result0 = lsvm0.predict(test_feat0)
result1 = lsvm1.predict(test_feat1)
result2 = lsvm2.predict(test_feat2)

print "test accuracy aspect ratio 0", lsvm0.score(test_feat0, test_label0)
print "test accuracy aspect ratio 1", lsvm1.score(test_feat1, test_label1)
print "test accuracy aspect ratio 2", lsvm2.score(test_feat2, test_label2)

print "first aspect ratio:"
printConfusionMatrix(result0, test_label0)

print "second aspect ratio:"
printConfusionMatrix(result1, test_label1)

print "third aspect ratio:"
printConfusionMatrix(result2, test_label2)


test accuracy aspect ratio 0 0.995260663507
test accuracy aspect ratio 1 0.986842105263
test accuracy aspect ratio 2 0.986046511628
first aspect ratio:
true positives: 12 true_negatives: 198 false positives: 0 false_negatives: 1 

second aspect ratio:
true positives: 31 true_negatives: 194 false positives: 1 false_negatives: 2 

third aspect ratio:
true positives: 16 true_negatives: 196 false positives: 0 false_negatives: 3 



Hard Negative Mining: scan negative images for false positives, store windows for retraining

In [ ]:
#get dataset of negative images to scan through
neg_files = gs.getAllFiles("hwy_empty")
neg_imgs = []
false_pos_feat = []
signs = 0
nosigns = 0

for f in neg_files:
    neg_imgs.append([f, cv2.imread("hwy_empty/" + f)])

#checking contents
print len(neg_imgs)

#multiscale detect

import multiscale_detect as md
counter = 1
for row in neg_imgs:
    scales = md.pyramid(row[1], scale=1.5, minSize=(30, 30))
    winh = 80
    winw = 80
     
    for img in scales:
        results = []
 
        for (x, y, window) in md.sliding_window(img, 40, (winw, winh)):        
            if window.shape[0] != winh or window.shape[1] != winw:
                continue
            window = cv2.resize(window, (80, 80))
            feat = hog.compute(window)
            result = lsvm.predict(feat.reshape(1,-1))[0]
            if result == 1:  
                false_pos_feat.append(feat)
                signs +=1
            else:
                nosigns+=1
    counter +=1


print "signs", signs, "no signs", nosigns
false_pos_feat = [x.flatten() for x in false_pos_feat] 
print len(false_pos_feat)

In [ ]:
false_pos_labels = [0] * len(false_pos_feat)


In [ ]:
#retrain classifier with false positives from hard negative mining
train_features_wfp = np.r_[train_features, np.array(false_pos_feat)]
train_labels_wfp = np.r_[train_labels, np.array(false_pos_labels)]
print train_features_wfp.shape, train_labels_wfp.shape

lsvm.fit(train_features_wfp, train_labels_wfp)
print "training accuracy", lsvm.score(train_features_wfp, train_labels_wfp)
#compute test accuracy
result = lsvm.predict(test_features)
print "test accuracy", lsvm.score(test_features, test_labels)
result = [int(x) for x in result]

false_positives = 0
false_negatives = 0
true_positives = 0
true_negatives = 0

false_pos_filenames = []
false_neg_filenames = []
true_pos_filenames = []
for i, row in enumerate(test_labels):
    if row  == 1 and result[i] == 1: 
        true_positives +=1
    if row  != 1 and result[i] != 1: 
        true_negatives +=1
    if row  != 1 and result[i] == 1: 
        false_positives +=1
    if row == 1 and result[i] != 1: 
        false_negatives +=1	

print "true positives:", true_positives, "true_negatives:", true_negatives, "false positives:", false_positives, "false_negatives:", false_negatives, "\n"
# print "false positives", false_pos_filenames, "\n"
# print "false negatives", false_neg_filenames, "\n"




Find and display signs in a test image

In [ ]:
def transform_scale(factor, boxes):
    return [(int(x1*factor), int(y1*factor), int(x2*factor), int(y2*factor)) for (x1,y1,x2,y2) in boxes]


import multiscale_detect as md
from random import randint
color = (0,255,0)
clone = None
dir = "multiscale_test"
files = os.listdir(dir)
for i in range(len(files)):
    test_image = cv2.imread(dir+"/"+files[i])
    if test_image != None and test_image.any():
        test_image = imutils.resize(test_image, width=min(600, test_image.shape[1]))
        clone = test_image.copy()
        clone = cv2.cvtColor(clone, cv2.COLOR_BGR2RGB)

        pscale = 1.1
        scales = md.pyramid(test_image, scale=pscale, minSize=(20, 20))
        winh = 54
        winw = 80
        scaled_results = []

        for i, img in enumerate(scales):
            results = []

            for (x, y, window) in md.sliding_window(img, 20, (winw, winh)):        
                if window.shape[0] != winh or window.shape[1] != winw:
                    continue
                wclone = img.copy()
                wclone = cv2.cvtColor(wclone, cv2.COLOR_BGR2RGB)
                window = cv2.resize(window, (80, 80))
                feat = hog.compute(window)
                result = lsvm.predict(feat.reshape(1,-1))[0]
                if result == 1:
                    results.append((x, y, x+winw, y+winh))

            factor = float(clone.shape[0])/img.shape[0]   
            scaled = transform_scale(factor, results)
            scaled_results.extend(scaled)


        scaled_results = md.non_max_suppression_fast(np.array(scaled_results), .3)
        for x1, y1, x2, y2 in scaled_results:
            cv2.rectangle(clone, (x1, y1), (x2, y2), color, 2)

        plt.imshow(clone)
        plt.show()
    else: print "Image not found"


# 